In [14]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchtext
import io
from happytransformer import HappyTextClassification 


In [17]:
!nvidia-smi
print(torch.cuda.is_available())

Sun Jan 15 01:08:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.77       Driver Version: 512.77       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| 18%   40C    P8    17W / 215W |    738MiB /  8192MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
happy_tc = HappyTextClassification("DISTILBERT", "distilbert-base-uncased-finetuned-sst-2-english")
from happytransformer import TCTrainArgs
args = TCTrainArgs(batch_size=8)
happy_tc.train("train.csv", args=args)


Downloading: 100%|██████████| 629/629 [00:00<00:00, 127kB/s]
c:\Users\Melina\anaconda3\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Melina\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading: 100%|██████████| 268M/268M [00:06<00:00, 44.2MB/s] 
Downloading: 100%|███

In [11]:
print(happy_tc.classify_text("I am so happy about this!"))
print(happy_tc.classify_text("This is the most dissapointing news I have had in a while."))
print(happy_tc.classify_text("This house was built three years ago."))
print(happy_tc.classify_text("I am amazed by how amazing these nuts are!"))
print(happy_tc.classify_text("They have no milk, this sucks."))

TextClassificationResult(label='POSITIVE', score=0.9998737573623657)
TextClassificationResult(label='NEGATIVE', score=0.9828665852546692)
TextClassificationResult(label='NEGATIVE', score=0.9781290888786316)
TextClassificationResult(label='POSITIVE', score=0.9998642206192017)
TextClassificationResult(label='NEGATIVE', score=0.9990074038505554)


In [14]:
#model = fasttext.train_unsupervised('data/enwik9')
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data



In [9]:
class sentiment_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_class):
        super(TweetRNN, self).__init__()
        self.emb = nn.Embedding.from_pretrained(glove.vectors) # Replace with embeddings
        self.hidden_size = hidden_size
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_class)

    def forward(self, x): 
        # Look-up the embeddings 
        x = self.emb(x) 
        # Set the initial hidden states 
        h0 = torch.zeros(1, x.size(0), self.hidden_size) 
        c0 = torch.zeros(1, x.size(0), self.hidden_size) 
        # Forward propagate the RNN 
        out, __ = self.rnn(x, (h0, c0)) 
        # Pass the output of the last step to the classifier return 
        self.fc(out[:,-1,:])

